In [ ]:
!playwright install chromium

In [2]:
from playwright.async_api import async_playwright
import pandas as pd
import asyncio

In [3]:

# Function to validate the ticker using Playwright
async def validate_ticker(page, ticker):
    """
    Validates if the ticker exists by navigating to its page and checking the response.
    """
    url = f'https://www.screener.in/company/{ticker}/consolidated/'
    response = await page.goto(url)
    
    if response.status == 200:
        print(f"Ticker '{ticker}' is valid. Proceeding...")
        return True
    elif response.status == 404:
        print(f"Ticker '{ticker}' does not exist. Please check the ticker name.")
        return False
    else:
        print(f"Failed to fetch the page. Status code: {response.status}")
        return False

# Function to extract table data
async def extract_table_data(page, section_id):
    """
    Extracts data from a table in the specified section asynchronously.
    """
    try:
        
        # Locate the section by ID
        section = page.locator(f"section#{section_id}")
        table = section.locator("table.data-table")

        # Extract headers
        headers = await table.locator("thead th").all_inner_texts()
        
        # Expand all buttons to reveal hidden rows
        buttons = section.locator("button.button-plain")
        for button in await buttons.all():
            await button.click()
            await page.wait_for_timeout(500)  # Wait briefly for data to load

        # Extract rows
        rows = table.locator("tbody tr")
        data = []
        for row in await rows.all():
            cells = await row.locator("td").all_inner_texts()
            data.append(cells)

        # Create DataFrame
        df = pd.DataFrame(data, columns=headers)
        df.set_index(df.columns[0], inplace=True)
        for col in df.columns[0:]:
            df[col] = pd.to_numeric(df[col].str.replace(",", "").str.replace("%", ""), errors="coerce")
        # Strip spaces and normalize the index
        df.index = df.index.str.strip().str.replace(r'\s*-\s*$', '', regex=True)

        return df
        return df
          
    except Exception as e:
        print(f"Error extracting data from section {section_id}: {e}")
        return pd.DataFrame()

# Main function to validate the ticker and extract data
async def main(ticker):
    async with async_playwright() as p:
        # Launch browser
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()

        # Validate ticker
        if not await validate_ticker(page, ticker):
            await browser.close()
            return  # Exit if the ticker is invalid

        # Extract tables
        pnl_df = await extract_table_data(page, "profit-loss")
        balance_sheet_df = await extract_table_data(page, "balance-sheet")
        cash_flow_df = await extract_table_data(page, "cash-flow")
        
        # Extract Market Cap
        market_cap_selector = 'ul#top-ratios > li:nth-child(1) .number'
        market_cap_str = await page.locator(market_cap_selector).inner_text()
        
        # Extract Current Price
        cmp_selector = 'ul#top-ratios > li:nth-child(2) .number'
        cmp_str = await page.locator(cmp_selector).inner_text()

        market_cap = float(market_cap_str.replace(',', ''))
        cmp = float(cmp_str.replace(',',''))
        print(f"market cp= {market_cap}")
        print(f"CMP= {cmp}")

        # Print extracted DataFrames
        print("\nProfit and Loss DataFrame:")
        print(pnl_df)
        print("\nBalance Sheet DataFrame:")
        print(balance_sheet_df)
        print("\nCash Flow DataFrame:")
        print(cash_flow_df)
        return pnl_df,balance_sheet_df,cash_flow_df,market_cap,cmp
        # Close browser
        await browser.close()

if __name__ == "__main__":
    stock_name = input("Enter screener ticker name: ")
    
    pnl_df, balance_sheet_df, cash_flow_df, market_cap,cmp = await main(stock_name)


Enter screener ticker name:  TCS


Ticker 'TCS' is valid. Proceeding...
market cp= 1502357.0
CMP= 4152.0

Profit and Loss DataFrame:
                      Mar 2013  Mar 2014  Mar 2015   Mar 2016   Mar 2017  \
                                                                           
Sales                 62989.00  81809.00  94648.00  108646.00  117966.00   
Sales Growth %           28.83     29.88     15.69      14.79       8.58   
Expenses              44950.00  56657.00  70167.00   77969.00   85655.00   
Material Cost %           0.00      0.00      0.00       0.00       0.00   
Raw material cost         0.00      0.00      0.00       0.00      94.00   
Change in inventory       0.00      0.00      0.00       0.00       1.00   
Manufacturing Cost %     26.00     26.00     26.00       2.00       2.00   
Employee Cost %          38.00     36.00     41.00      51.00      52.00   
Other Cost %              7.00      7.00      7.00      18.00      18.00   
Operating Profit      18040.00  25153.00  24482.00   30677.00   32

In [4]:
pnl_df.fillna(0, inplace=True)
balance_sheet_df.fillna(0, inplace=True)
cash_flow_df.fillna(0, inplace=True)


In [26]:
print(balance_sheet_df.index)

Index(['Equity Capital', 'Reserves', 'Borrowings', 'Short term Borrowings',
       'Other Liabilities', 'Trade Payables', 'Other liability items',
       'Total Liabilities', 'Fixed Assets', 'Computers',
       'Furniture n fittings', 'Vehicles', 'Other fixed assets', 'Gross Block',
       'Accumulated Depreciation', 'CWIP', 'Investments', 'Other Assets',
       'Inventories', 'Trade receivables', 'Cash Equivalents',
       'Loans n Advances', 'Other asset items', 'Total Assets'],
      dtype='object', name='')


In [5]:
Operating_Cf = pd.to_numeric(cash_flow_df.loc["Cash from Operating Activity"])
Fixed_Asset= cash_flow_df.loc["Fixed assets purchased"]

FCF=Operating_Cf+Fixed_Asset
Avg_Fcf=FCF[-5:].mean()
Avg_Fcf = Avg_Fcf if isinstance(Avg_Fcf, (int, float)) else Avg_Fcf.iloc[0]

print(FCF)
print(Avg_Fcf)

Mar 2013     8977
Mar 2014    11625
Mar 2015    16420
Mar 2016    17122
Mar 2017    23234
Mar 2018    23205
Mar 2019    26362
Mar 2020    29120
Mar 2021    35626
Mar 2022    36954
Mar 2023    38865
Mar 2024    41664
dtype: int64
36445.8


In [6]:
estimated_growth_rate_1= input("What is the estimated growth rate for the first 5 years? (Default: 15%) ")

if not estimated_growth_rate_1.strip():
    estimated_growth_rate_1=15
else:
    estimated_growth_rate_1=float(estimated_growth_rate_1)

Forecasted_FCF_1=[]
# Loop to calculate percentage change
for i in range(1, 6):  # Start from index 1
    perc_change=Avg_Fcf*(1+(estimated_growth_rate_1/100))
    Forecasted_FCF_1.append(perc_change)  # Append the result to the list
    Avg_Fcf=perc_change
print(Forecasted_FCF_1)


What is the estimated growth rate for the first 5 years? (Default: 15%)  12


[40819.29600000001, 45717.61152000001, 51203.72490240002, 57348.17189068803, 64229.95251757059]


In [7]:
estimated_growth_rate_2= input("What is the estimated growth rate for the second 5 years? (Default: 10%) ")

if not estimated_growth_rate_2.strip():
    estimated_growth_rate_2=10
else:
    estimated_growth_rate_2=float(estimated_growth_rate_1)


# Loop to calculate percentage change
for i in range(1, 6):  # Start from index 1
    perc_change=Avg_Fcf*(1+(estimated_growth_rate_2/100))
    Forecasted_FCF_1.append(perc_change)  # Append the result to the list
    Avg_Fcf=perc_change
print(Forecasted_FCF_1)

What is the estimated growth rate for the second 5 years? (Default: 10%)  7


[40819.29600000001, 45717.61152000001, 51203.72490240002, 57348.17189068803, 64229.95251757059, 71937.54681967908, 80570.05243804057, 90238.45873060545, 101067.0737782781, 113195.12263167149]


In [8]:
terminal_rate= input("What is the terminal growth rate? (Default: 3.5%) ")
risk_free_rate=float(input("What is the current risk free rate? (in %)(reccomendation: Use 91days T-bill yield) "))

if not terminal_rate.strip():
    terminal_rate=3.5/100
else:
    terminal_rate=float(terminal_rate)

terminal_rate=terminal_rate/100
risk_free_rate=risk_free_rate/100


TV=Avg_Fcf*(1+terminal_rate)
(risk_free_rate-terminal_rate)
Forecasted_FCF_1.append(TV)  # Append the result to the list
print(Forecasted_FCF_1)

What is the terminal growth rate? (Default: 3.5%)  2.0
What is the current risk free rate? (in %)(reccomendation: Use 91days T-bill yield)  6.63


[40819.29600000001, 45717.61152000001, 51203.72490240002, 57348.17189068803, 64229.95251757059, 71937.54681967908, 80570.05243804057, 90238.45873060545, 101067.0737782781, 113195.12263167149, 115459.02508430491]


In [9]:
present_value_fcf=[]

risk_free_rate=float(input("What is the current risk free rate? (in %)(reccomendation: Use 91days T-bill yield) "))

for i in range(0, 10):  # Start from index 1
    pv=Forecasted_FCF_1[i]/((1+(risk_free_rate/100))**(i+1))
    present_value_fcf.append(pv)


pv_tv=pv=Forecasted_FCF_1[-1]/((1+(risk_free_rate/100))**(10))
present_value_fcf.append(pv_tv)

print(present_value_fcf)

net_pv=sum(present_value_fcf)
print(net_pv)

What is the current risk free rate? (in %)(reccomendation: Use 91days T-bill yield)  6.36


[38378.42798044378, 40413.53830208446, 42556.565342548514, 44813.231650671616, 47189.563227484214, 49691.90561750876, 52326.940853337546, 55101.7052987383, 58023.608437934265, 61100.452661232026, 62322.461714456666]
551918.4010864401


In [10]:
#Calculating net Debt

cash_equivalent=balance_sheet_df.loc["Cash Equivalents"].iloc[-1]
total_debt=balance_sheet_df.loc["Borrowings"].iloc[-1]

net_debt=total_debt-cash_equivalent
print(net_debt)

-7687.0


In [11]:
no_of_outstanding_shares=market_cap/cmp
print(no_of_outstanding_shares)

361.8393545279383


In [12]:
total_pv=net_pv-net_debt

intrinsic_value=total_pv/no_of_outstanding_shares
print(intrinsic_value)

1546.5575927099214
